In [1]:
import dew_gwdata as gd
import pandas as pd

In [2]:
db = gd.sageodata()

In [3]:
dh_nos = [104532, 342062, 275813]

In [4]:
df = db.query(
    db.SQL(
        """
SELECT wp.permit_no AS permit_no_only,
       To_char(con.permit_no)
              || To_char(con.permit_ex) AS permit_no_full,
       wp.wilma_or_wls                  AS permit_data_source,
       wp.issue_date                    AS permit_issued,
       wp.expiry_date                   AS permit_expiry,
       wp.purpose                       AS permit_purpose,
       wp.well_use                      AS permit_well_use,
       wp.holder_name                   AS permit_holder,
       wp.comments                      AS permit_comments,
       To_char(wp.holder_address_1
              || '\n'
              || wp.holder_address_2
              || '\n'
              || wp.holder_suburb
              || ' '
              || wp.holder_state
              || ' '
              || wp.holder_postcode) AS permit_holder_address,
       wp.area_desc                  AS permit_area_desc,
       wp.multiple_non_bkf_wells_flag,
       con.drillhole_no AS dh_no,
       dh.unit_no       AS unit_long,
       To_char(dh.map_100000_no)
              || '-'
              || To_char(dh.dh_seq_no) AS unit_hyphen,
       Trim(To_char(dh.obs_well_plan_code))
              || Trim(To_char(dh.obs_well_seq_no, '000')) AS obs_no,
       dh.dh_name,
       summ.aq_subaq AS aquifer,
       con.completion_date,
       con.completion_no,
       con.completion_date,
       con.start_depth,
       con.total_dpth                 AS total_depth,
       con.current_dpth               AS current_depth,
       con.final_dpth                 AS final_depth,
       con.final_standing_water_level AS final_swl,
       con.final_well_yield           AS final_yield,
       con.comments,
       con.screened,
       con.pcem                 AS pcemented,
       con.dev                  AS developed,
       con.abndn_ind            AS abandoned,
       con.bkf_ind              AS backfilled,
       con.dry_ind              AS dry,
       con.enlarged_ind         AS enlarged,
       con.flowing_ind          AS flowing,
       con.replacement_well_ind AS replacement,
       con.commenced_date,
       con.created_by      AS construction_created_by,
       con.creation_date   AS construction_creation_date,
       con.modified_by     AS construction_modified_by,
       con.modified_date   AS construction_modified_date,
       dh.amg_easting      AS easting,
       dh.amg_northing     AS northing,
       dh.amg_zone         AS zone,
       dh.neg_lat_deg_real AS latitude,
       dh.long_deg_real    AS longitude
FROM   (
              SELECT subwp.*
              FROM   dhdbview.wp_permit_wls_vw subwp
              join   dhdb.dc_construction_vw subcon
              ON     subcon.permit_no = subwp.permit_no
              WHERE  subcon.drillhole_no IN {DH_NO} ) wp
join   dhdb.dc_construction_vw con
ON     wp.permit_no = con.permit_no
join   dhdb.dd_drillhole_vw dh
ON     con.drillhole_no = dh.drillhole_no
join   dhdb.dd_drillhole_summary_vw summ
ON     dh.drillhole_no = summ.drillhole_no
""",
        dh_no=dh_nos,
    )
)
# where dh.drillhole_no in {DH_NO}
# dhdbview.wp_permit_wls_vw wp

with pd.option_context(
    "display.max_rows", 1000, "display.width", 1000, "display.max_colwidth", 20
):
    print(df.T)

                                        0                    1                    2                    3                    4                    5
permit_no_only                        393                  393               220805               252536               362998               362998
permit_no_full                        393                  393               220805               252536               362998               362998
permit_data_source                    WLS                  WLS                WILMA                WILMA                WILMA                WILMA
permit_issued         1976-09-01 00:00:00  1976-09-01 00:00:00  2013-04-09 00:00:00  2015-09-10 00:00:00  2020-05-11 00:00:00  2020-05-11 00:00:00
permit_expiry         1977-09-01 00:00:00  1977-09-01 00:00:00  2014-04-09 00:00:00  2016-09-10 00:00:00  2021-05-11 00:00:00  2021-05-11 00:00:00
permit_purpose        TO CONSTRUCT A W...  TO CONSTRUCT A W...    Well Construction        Well Backfill    Well Const

In [20]:
db.wells_summary(dh_nos)[["dh_no", "dh_name"]]

,dh_no,dh_name
0,104532,NAMUR NO 1
1,275813,MW 5
2,342062,DH-18-T


In [21]:
db.permits_by_completed_drillholes_only(dh_nos)[
    ["permit_no_full", "dh_no", "permit_holder", "permit_purpose"]
]

,permit_no_full,dh_no,permit_holder,permit_purpose
0,393,104532,DELHI INTERNATIONAL OIL CORP,TO CONSTRUCT A WATER SUPPLY WELL
1,220805,275813,CALTEX AUSTRALIA PETROLEUM PTY LTD,Well Construction
2,252536,275813,WSP PARSONS BRINCKERHOFF,Well Backfill
3,362998,342062,MINISTER FOR ENVIRONMENT AND WATER,Well Construction


In [22]:
db.permits_by_completed_drillholes_all(dh_nos)[
    ["permit_no_full", "dh_no", "permit_holder", "permit_purpose"]
]

,permit_no_full,dh_no,permit_holder,permit_purpose
0,393,7429,DELHI INTERNATIONAL OIL CORP,TO CONSTRUCT A WATER SUPPLY WELL
1,393,104532,DELHI INTERNATIONAL OIL CORP,TO CONSTRUCT A WATER SUPPLY WELL
2,220805,275813,CALTEX AUSTRALIA PETROLEUM PTY LTD,Well Construction
3,252536,275813,WSP PARSONS BRINCKERHOFF,Well Backfill
4,362998,342062,MINISTER FOR ENVIRONMENT AND WATER,Well Construction
5,362998,348967,MINISTER FOR ENVIRONMENT AND WATER,Well Construction
